# Intro

## Standard modules

In [1]:
import os, sys, pickle
import pathlib
import numpy as np
from tqdm.auto import tqdm, trange

In [2]:
from sentence_transformers import util

## My modules

In [3]:
from toolbox.updater import time_is_now

In [4]:
from file2text import file2text

In [5]:
from first_rough_cleaning import first_text_cleaner

In [6]:
from embed_me_II_second_wipe import file2vecfile

/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [7]:
from jina4lote import jina4lote

In [18]:
from embed_me_II import file2vecfile

## Alternatives

In [8]:
import pymupdf4llm

## Parameters

In [9]:
N_RUNS=10

## Files

In [14]:
PDF_FOLDER='./pdf/'

In [15]:
pdfs=os.listdir(PDF_FOLDER)
pdfs.sort()

In [19]:
file2vecfile(pdfs[0], extension='.pdf')

'2021_10_VODAFONE_GROUP_PLC.txt'

# Check a set of random .pdf file

In [ ]:
for pdf in tqdm(pdfs):
    md_text = pymupdf4llm.to_markdown(PDF_FOLDER+pdf)
    file_name=file2vecfile(pdfs[0], extension='.pdf').replace('.txt', '.md')
    pathlib.Path(PDF_FOLDER+file_name).write_bytes(md_text.encode())

  0%|          | 0/5 [00:00<?, ?it/s]

Processing ./pdf/10_Telecommunications_VODAFONE GROUP PLC_2021.pdf...
[                                        ] (0/2[=                                       ] ( 1/2=[===                                     ] ( 2/2=[=====                                   ] ( 3/2[======                                  ] ( 4/23=[========                                ] ( 5/23=[==========                              ] ( 6/23=[============                            ] ( 7/23[=============                           ] ( 8/2=[===============                         ] ( 9/2=[=================                       ] (10/2=[===================                     ] (11/2[====================                    ] (12/23=[======================                  ] (13/23=[========================                ] (14/23=[==========================              ] (15/23[===========================             ] (16/2=[=============================           ] (17/2=[===============================         ] (18